In [1]:
import pandas as pd
from datetime import timedelta
import numpy as np
import datetime
from sklearn.preprocessing import MinMaxScaler

In [2]:
def get_data():
    df = pd.read_csv('south_central_moisture_data_1988_2017.csv')
    df = df[['final_date', 'soil_moisture']]
    mmscaler = MinMaxScaler(feature_range=(0, 1))
    mmscaler.fit(df[['soil_moisture']])
    df.soil_moisture = mmscaler.transform(df[['soil_moisture']])
    df.final_date = pd.to_datetime(df.final_date)
    df = df.set_index('final_date')
    return df

In [3]:
df = get_data()

In [4]:
df.head()

,soil_moisture
final_date,
1988-03-01,0.598248
1988-03-08,0.583229
1988-03-15,0.529412
1988-03-22,0.698373
1988-03-29,0.624531


In [5]:
def agg_data(df):
    df_agg = df.soil_moisture.resample('M').agg('mean')
    df_agg = df_agg['1997':]
    df_agg = df_agg.to_frame(name=None)
    return df_agg

In [6]:
df_agg = agg_data(df)

In [7]:
df_agg.head()

,soil_moisture
final_date,
1997-01-31,0.557697
1997-02-28,0.766792
1997-03-31,0.948436
1997-04-30,0.989675
1997-05-31,0.977222


In [8]:
def find_outliers(df):
    agg_mean = df_agg.soil_moisture.mean()
    agg_stdev = df_agg.soil_moisture.std()
    df_agg["lower_bound"] = (agg_mean - (4*agg_stdev))
    df_agg["sm_lb"] = (df_agg.soil_moisture - df_agg.lower_bound)
    df_agg['outside_normal'] = (df_agg.sm_lb < 1)
    return df_agg

In [9]:
df_agg = find_outliers(df_agg)

In [16]:
csvname = 'four_week_soil_moisture_predictions.csv'
def clean_predictions(csv_name):
    predictions = pd.read_csv(csv_name)
    predictions.rename(columns={'date':'final_date'}, inplace=True)
    predictions.rename(columns={'predicted_soil_moisture':'soil_moisture'}, inplace=True)
    mmscaler = MinMaxScaler(feature_range=(0, 1))
    mmscaler.fit(predictions[['soil_moisture']])
    predictions.soil_moisture = mmscaler.transform(predictions[['soil_moisture']])
    predictions.final_date = pd.to_datetime(predictions.final_date)
    predictions = predictions.set_index('final_date')
    return predictions

In [20]:
clean_predictions()

,soil_moisture
final_date,
2018-01-07,0.000000
2018-01-14,1.000000
2018-01-21,0.783259
2018-01-28,0.837176
